# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [10]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_csv = "weather_data.csv"
city_data = pd.read_csv(city_csv)
city_data.head()

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
0,Nome,US,64.5011,-165.4064,-0.40,42,1,11.50
1,Albany,US,42.6001,-73.9662,30.00,71,100,3.42
2,Butaritari,KI,3.0707,172.7902,80.94,83,16,16.33
3,Chimbote,PE,-9.0853,-78.5783,75.67,69,23,10.31
4,Bluff,NZ,-46.6000,168.3333,52.39,90,100,13.06


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [36]:
# Configure gmaps
gmaps.configure(api_key=g_key)

figure_layout = {
    'width': '600px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px'
}

# create variables for heatmap
locations = city_data[["Latitude", "Longitude"]]
humidity = city_data["Humidity"].astype(float)
maxhumidity = humidity.max()

In [37]:
# API DAY 3 ACTIVITY 7: AIRPORT MAP
earth_center = (0, 0)
fig = gmaps.figure(layout=figure_layout, center=earth_center, zoom_level=1)

# create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=maxhumidity, point_radius=3)

# add layer
fig.add_layer(heat_layer)

# display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# DROP NULL VALUES: PANDAS DAY 2 ACTIVITY 4 (PORTLAND CRIME CLEANING)
vacation_df = pd.DataFrame(city_data.loc[(city_data["Max Temp"] >= 70) & (city_data["Max Temp"] <= 80) & (city_data["Cloudiness"] == 0), :])
vacation_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed
23,San Isidro,PE,-12.1167,-77.0500,77.00,73,0,8.05
34,Cape Town,ZA,-33.9258,18.4232,71.01,78,0,24.16
58,Chui,UY,-33.6971,-53.4616,72.46,75,0,9.10
85,Lima,PE,-12.0432,-77.0282,77.00,73,0,8.05
94,Pisco,PE,-13.7000,-76.2167,78.80,61,0,14.97
131,Broken Hill,AU,-31.9500,141.4333,71.60,28,0,6.91
157,Chincha Alta,PE,-13.4099,-76.1323,78.80,61,0,14.97
224,Porlamar,VE,10.9580,-63.8491,77.94,71,0,16.87
244,East London,ZA,-33.0153,27.9116,72.03,77,0,3.96
252,Ancud,CL,-41.8697,-73.8203,78.80,41,0,10.36


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [41]:
# create new hotel dataframe w/ city, country, coordinates & hotel name
hotel_df = pd.DataFrame(vacation_df[["City", "Country", "Latitude", "Longitude"]])

# add column to hold hotel info
hotel_df["Hotel Name"] = ""
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
23,San Isidro,PE,-12.1167,-77.0500,
34,Cape Town,ZA,-33.9258,18.4232,
58,Chui,UY,-33.6971,-53.4616,
85,Lima,PE,-12.0432,-77.0282,
94,Pisco,PE,-13.7000,-76.2167,
131,Broken Hill,AU,-31.9500,141.4333,
157,Chincha Alta,PE,-13.4099,-76.1323,
224,Porlamar,VE,10.9580,-63.8491,
244,East London,ZA,-33.0153,27.9116,
252,Ancud,CL,-41.8697,-73.8203,


In [61]:
# set parameters dictionary
params = {
    "type": "lodging",
    #"keyword": "hotel",
    "radius": 5000,
    #"rankby": "distance",
    "key": g_key
}

# Google API search
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# Run request
# use lat/lon & iterrows to iterate through pandas df
for index, row in hotel_df.iterrows():
    lat = row["Latitude"]
    long = row["Longitude"]
    city = row["City"]
    # add location parameter using coordinates for each row
    params["location"] = f"{lat},{long}"

    response = requests.get(base_url, params=params).json()

#print(json.dumps(response, indent=4, sort_keys=True))

    
    try: 
        print(f"Closest hotel to {city} is {response['results'][0]['name']}.")    
        hotel_df.loc[index, 'Hotel Name'] = response['results'][0]['name']
    except (KeyError, IndexError):
        print("No hotels found...skipping.")

Closest hotel to San Isidro is Radisson Hotel Decapolis Miraflores.
Closest hotel to Cape Town is Radisson Blu Hotel Waterfront, Cape Town.
Closest hotel to Chui is Nuevo Hotel Plaza.
Closest hotel to Lima is Gran Hotel Bolivar Lima.
Closest hotel to Pisco is Hotel San Isidro Oficial.
Closest hotel to Broken Hill is Royal Exchange Hotel.
Closest hotel to Chincha Alta is Casa Andina Standard Chincha.
Closest hotel to Porlamar is Hotel Bella Vista.
Closest hotel to East London is Tu Casa.
Closest hotel to Ancud is Hotel Arena Gruesa.
Closest hotel to Salalah is HAMDAN PLAZA HOTEL SALALAH.
Closest hotel to Arica is Hostal Jardin Del Sol.
Closest hotel to Río Gallegos is Hotel Santa Cruz.
Closest hotel to Ratnagiri is Hotel Landmark.
Closest hotel to Bilma is Expeditions Ténére voyages.
Closest hotel to Punta Alta is Center Hotel Punta Alta.


In [62]:
hotel_df

,City,Country,Latitude,Longitude,Hotel Name
23,San Isidro,PE,-12.1167,-77.0500,Radisson Hotel Decapolis Miraflores
34,Cape Town,ZA,-33.9258,18.4232,"Radisson Blu Hotel Waterfront, Cape Town"
58,Chui,UY,-33.6971,-53.4616,Nuevo Hotel Plaza
85,Lima,PE,-12.0432,-77.0282,Gran Hotel Bolivar Lima
94,Pisco,PE,-13.7000,-76.2167,Hotel San Isidro Oficial
131,Broken Hill,AU,-31.9500,141.4333,Royal Exchange Hotel
157,Chincha Alta,PE,-13.4099,-76.1323,Casa Andina Standard Chincha
224,Porlamar,VE,10.9580,-63.8491,Hotel Bella Vista
244,East London,ZA,-33.0153,27.9116,Tu Casa
252,Ancud,CL,-41.8697,-73.8203,Hotel Arena Gruesa


In [65]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [66]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', padding='1px', width='600px'))